# Installing Dependencies

In [3]:
using DelimitedFiles
using NearestNeighbors: KDTree, knn
using LinearAlgebra: norm
using CUDA
using BenchmarkTools: @btime

# Using kNN

In [4]:
@time function NearNeighbor(X, t, t_knn)
    # Using kNN for Nearest Neighbors
    if t%t_knn | t == 0
        kdtree = KDTree(X')
    end 

    # Loop over all cells to compute displacements
    index, _ = knn(kdtree, X[1,:], 14, true)
    for i in 2:size(X)[1]
        # Scan neighbours
        global idxs, _ = knn(kdtree, X[i,:], 14, true)
        index = hcat(index, idxs)

    end
    return index
end

  0.002953 seconds (996 allocations: 64.106 KiB, 52.80% compilation time)


NearNeighbor (generic function with 1 method)

# Initial Conditions

In [354]:
R_Agg = 16
t, t_knn = 0, 10
r_max, s = 4, 1
K = 1

@time X = readdlm("../data/Init/Two_Sphere/$R_Agg.csv", ',', Float64, header=true)[1][:, 1:3]
@time idx = NearNeighbor(X, t, t_knn)

14×6064 Matrix{Int64}:
   1    2    3    4    5    6    7  …  6059  6060  6061  6062  6063  6064
  25   26   27   40   42   43   44     6024  6025  6026  6038  6039  6040
  33   33   35   41   41   42   43     6023  6024  6025  6030  6031  6033
  32   34   34    9    9   10   11     6055  6057  6057  6031  6032  6032
   2    3    2    8   10   11    6     6056  6056  6060  6063  6064  6063
   4    1    6    5    4    5   35  …  6058  6061  6033  6059  6062  6061
   5    6    7   32    6    7    3     6060  6059  6064  6058  6060  6060
  24    5   26    1   33   34   51     6031  6032  6017  6037  6059  6039
  26   25   28   48    2    3   34     6062  6063  6034  6039  6038  6041
  41   27   43   33    1    2   36     6063  6064  6032  6023  6040  6025
 102   42  104   31   49   50  114  …  6015  6016  5954  5962  6024  5964
  20  103   22  111   34   35   52     6030  6033  6018  6045  5963  6047
  19   20   21   47   32   33   50     6032  6031  6016  6044  6045  6046
  42   21   44 

# CUDA Forces

## Define Variables for CUDA

In [363]:
@time X_Cu = X |> cu
@time dX_Cu = zeros(size(X)[1],3) |> cu
@time idx_Cu = idx |> cu
@time r = zeros(14,size(X)[1],3) |> cu
@time dist = zeros(14, size(X)[1]) |> cu
@time F = zeros(14, size(X)[1],3) |> cu;

# Joining the Function

In [251]:
function main()
    global X_Cu
    global dX_Cu
    global idx_Cu
    global r 
    global dist

    # Finding Distances
    println("Finding Distances")
    @time r = reshape(repeat(X_Cu, inner=(14,1)), 14, size(X_Cu)[1], 3) - X_Cu[idx_Cu,:]

    # Finding Distances/Norm
    println("Finding Distances/Norm")
    @time dist = (r[:,:,1] .^ 2 + r[:,:,2] .^ 2 + r[:,:,3] .^ 2) .^ (0.5)
    @time dist = reshape(repeat((dist), outer=(1,3)) ,14 ,size(X_Cu)[1], 3)

    # Deleting Distances greater than R_Max
    println("Deleting Distances greater than R_Max")
    r[dist .>  r_max] .= 0
    
    # Normalizationg Distances
    println("Normalizationg Distances")
    r = r ./ dist

    # dX[i,:] +=  r/dist * F
    println("Calculationg Forces")
    @time F = (-K.*(dist .- r_max).^2) .* (dist .- s) .* r

    # Calculating de dX
    println("Calculationg dX")
    @time dX_Cu[:,1] = sum(F[:,:,1][2:end,:]; dims=1)
    @time dX_Cu[:,2] = sum(F[:,:,2][2:end,:]; dims=1)
    @time dX_Cu[:,3] = sum(F[:,:,3][2:end,:]; dims=1)

    # Euler Step for all Cells
    X_Cu = X_Cu + dX_Cu
end

main (generic function with 1 method)

In [366]:
F = (-K.*(dist .- r_max).^2) .* (dist .- s) .* r

14×6064×3 CuArray{Float64, 3, CUDA.Mem.DeviceBuffer}:
[:, :, 1] =
 NaN        NaN        NaN        …  NaN        NaN        NaN
  -0.0       -0.0       -0.0          -0.0       -0.0       -0.0
   2.00167   -2.00167    2.00167      -2.00167   -2.00167    2.00167
  -2.00167    2.00167   -2.00167       2.00167    2.00167   -2.00167
   4.0        4.0       -4.0           4.0        4.0       -4.0
  -1.99306   -4.0       -1.99306  …    1.99306   -4.0        1.99306
   1.99306    1.99306    1.99306      -1.99306    1.99306   -1.99306
  -1.7844    -1.99306   -1.7844       -1.7844    -1.99306   -1.7844
   1.7844    -1.7844     1.7844        1.7844    -1.7844     1.7844
  -0.0        1.7844    -0.0          -0.0        1.7844    -0.0
  -0.0       -0.0       -0.0      …   -0.0       -0.0       -0.0
   0.20951   -0.0        0.20951       0.20951   -0.0        0.20951
  -0.20951   -0.20951   -0.20951      -0.20951   -0.20951   -0.20951
   0.41006    0.20951    0.41006       0.41006    0.20951    

In [193]:
main()
dX_Cu

Finding Distances
  0.000171 seconds (54 allocations: 2.438 KiB)
Finding Distances/Norm
  0.000191 seconds (176 allocations: 7.422 KiB)
Deleting Distances greater than R_Max
  0.000390 seconds (276 allocations: 12.609 KiB)
  0.000172 seconds (276 allocations: 12.609 KiB)
  0.000155 seconds (276 allocations: 12.609 KiB)
Calculationg Forces
  0.000075 seconds (79 allocations: 7.562 KiB)
  0.000018 seconds (22 allocations: 928 bytes)
  0.000012 seconds (22 allocations: 928 bytes)
  0.000011 seconds (22 allocations: 928 bytes)
Calculationg dX
  0.000062 seconds (72 allocations: 3.125 KiB)
  0.000039 seconds (72 allocations: 3.125 KiB)
  0.000038 seconds (72 allocations: 3.125 KiB)


6064×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  0.0087109    -7.22115f-5    0.0120923
 -0.000293967  -0.000382664   0.133869
 -0.00891659   -8.64309f-5    0.0125406
 -0.0402018     0.0248049     0.101925
  0.0475429     0.0189104     0.060228
 -0.0451324     0.0214268     0.0590496
  0.0861331     0.0550231     0.192301
 -0.993242     -0.241977      1.71085
  0.0914344     0.114845      0.106358
 -0.0955197     0.110076      0.108197
  ⋮                          
 -0.0957145    -0.110304     -0.108505
 -0.00817682   -0.00811106    4.5375f-5
 -0.0400234    -0.0246999    -0.101558
  0.047544     -0.018914     -0.0602371
 -0.0451159    -0.0214247    -0.0590313
  0.0864135    -0.0552018    -0.1928
  0.00857394    7.05692f-5   -0.0118797
 -0.000224217   0.000384661  -0.133939
 -0.00903928    8.85646f-5   -0.0127302